In [15]:
!pip install yfinance

In [29]:
# import libraries
import pandas as pd
import numpy as np
import os
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix

import boto3
import sagemaker

In [30]:
data_dir = 'stock_data'

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [31]:
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

In [32]:
# should be the name of directory you created to save your features data
data_dir = 'stock_data'

# set prefix, a descriptive name for a directory  
prefix = 'stock_prediction'

# upload all data to S3
inputdata = sagemaker_session.upload_data(data_dir, bucket=bucket, key_prefix=prefix)

In [33]:
# your import and estimator code, here

from sagemaker.sklearn.estimator import SKLearn

estimator = SKLearn(entry_point="train.py",
                    instance_type='ml.m4.xlarge',
                    source_dir="source_sklearn",
                    framework_version='0.20.0',
                    py_version="py3",
                    role=role,
                    instance_count=1,
                    sagemaker_session=sagemaker_session)


In [34]:
#Fit the model
# Train your estimator on S3 training data
estimator.fit({'train': inputdata})

2021-03-18 12:21:02 Starting - Starting the training job...
2021-03-18 12:21:26 Starting - Launching requested ML instancesProfilerReport-1616070062: InProgress
......
2021-03-18 12:22:26 Starting - Preparing the instances for training......
2021-03-18 12:23:26 Downloading - Downloading input data...
2021-03-18 12:23:51 Training - Downloading the training image...
2021-03-18 12:24:27 Uploading - Uploading generated training model2021-03-18 12:24:22,086 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2021-03-18 12:24:22,090 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-03-18 12:24:22,106 sagemaker_sklearn_container.training INFO     Invoking user training script.
2021-03-18 12:24:22,483 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-03-18 12:24:22,496 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-03-18 12:24:22,510 sagemaker-t


2021-03-18 12:24:46 Completed - Training job completed
Training seconds: 72
Billable seconds: 72


In [35]:
#Deploy the model
# deploy your model to create a predictor
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

---------------------!

In [36]:
# Test the model
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

In [37]:
# First: generate predicted, class labels
test_y_preds = predictor.predict(test_x)

In [42]:
# Second: calculate the test accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_y, test_y_preds)

print(accuracy)

0.5159362549800797


In [39]:
#Confusion matrix
confusion_matrix(test_y, test_y_preds)

array([[150, 135],
       [108, 109]])

In [40]:
#Cleanup 
predictor.delete_endpoint()

In [41]:
#Cleanup
bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'HRZ086K9QB63BR37',
   'HostId': 'KqqgZkcVy8Hee7B5NE76DXP3jbYoJvPzlDBEbh9KLC4C4Y/Px6HsktEt9TTeNVM7/AbsSjh0T1U=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'KqqgZkcVy8Hee7B5NE76DXP3jbYoJvPzlDBEbh9KLC4C4Y/Px6HsktEt9TTeNVM7/AbsSjh0T1U=',
    'x-amz-request-id': 'HRZ086K9QB63BR37',
    'date': 'Thu, 18 Mar 2021 12:35:49 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker-scikit-learn-2021-03-18-12-21-02-349/rule-output/ProfilerReport-1616070062/profiler-output/profiler-reports/OverallSystemUsage.json'},
   {'Key': 'sagemaker-scikit-learn-2021-03-18-12-21-02-349/source/sourcedir.tar.gz'},
   {'Key': 'sagemaker-scikit-learn-2021-03-18-12-21-02-349/rule-output/ProfilerReport-1616070062/profiler-output/profiler-reports/BatchSize.json'},
   {'Key': 'sagemaker-scikit-learn-2021-03-18-12-21-02-349/debug-out